# KEN3140 - Lab 3.1 
## Defining OWL ontologies in Protege. Running the reasoner, inferring new information, and debugging inconsistencies. 
- Install Protege (if not already done) - https://protege.stanford.edu/software.php

### Task 1: Creating some object properties for family relations (parentage)
1. Open protege and create a new ontology
2. Create an object property ```hasMother```
3. Create a property ```isMotherOf``` and set ```hasMother``` as ```InverseOf: isMotherOf```
4. Repeat for the property ```hasFather```;
5. Create a new property ```hasParent```; also create inverse properties ```hasChild``` and ```ifParentOf```
6. Make ```hasMother``` and ```hasFather``` sub-properties of ```hasParent```
7. Run the reasoner and look at the property hierarchy.

### Task 2: Assert some facts about individuals (create the ABox)
| **Person**                           | **hasFather**                                     | **hasMother**                                |
|--------------------------------------|--------------------------------------------------|----------------------------------------------|
| **Queen Elizabeth II**               | King George VI                                   | Queen Elizabeth The Queen Mother             |
| **Princess Margaret Countess of Snowdon** | King George VI                             | Queen Elizabeth The Queen Mother             |
| **Charles III**                      | Prince Philip Duke of Edinburgh                  | Queen Elizabeth II                          |
| **Anne Princess Royal**             | Prince Philip Duke of Edinburgh                  | Queen Elizabeth II                          |
| **William Prince of Wales**         | Charles III                                       | Diana Princess of Wales                    |
| **Prince Harry Duke of Sussex**     | Charles III                                       | Diana Princess of Wales                    |
| **Prince George of Wales**           | William Prince of Wales                          | Catherine Princess of Wales                |
| **Princess Charlotte of Wales**      | William Prince of Wales                          | Catherine Princess of Wales                |


### Task 3: Describing Ancestors and Descendants
1. Define a new object property ```hasRelation```. Set this as a *symmetric property*.
2. Define a new object property ```hasAncestor```.
3. Make ```hasAncestor``` a sub-property of ```hasRelation``` and a super-property of ```hasParent```. Set it as a *transitive property*.
4. Create the inverse of ```hasAncestor``` as ```hasDescendant```. 

### Task 4: Run DL Queries
1. Children of Queen Elizabeth II - ? (Tick under Query for - Instances)
2. Descendants of King George VI - ? 

### Task 5: Defining Grandparents and Great-grandparents --> OWL sub-property chains
Use definition: ```ObjectProperty: hasGrandparent SubPropertyChain: hasParent o hasParent```  
```
ObjectProperty: hasGrandParent
SubPropertyOf: hasAncestor
SubPropertyChain: hasParent o hasParent
SuperPropertyOf: hasGrandmother, hasGrandfather
InverseOf: isGrandParentOf
```
```
ObjectProperty: hasGrandFather
SubPropertyChain: hasParent o hasFather
```
```
ObjectProperty: hasGrandMother
SubPropertyChain: hasParent o hasMother
```

1. Define object properties ```hasGrandparent```, ```hasGrandMother``` and ```hasGrandFather```, and the inverses
2. Use the Superproperty Of (Chain) to define ```hasParent o hasParent```
3. Run DL Query - ```hasGrandParent value King_George_VI```

Similarly for great-grandparents. 
```
ObjectProperty: hasGreatGrandParent
SubPropertyChain: hasParent o hasParent o hasParent
```


Up until now, we have not defined any classes. We will now define the ```Person``` and the ```Gender``` class, and use these to also define *domain* and *range* for many of the properties defined earlier. 

### Task 6: Define the `Person` and `Gender` classes. 
1. Create a class `Person`.
2. Create another class `Gender`.
3. Make `Person` and `Gender` disjoint.
4. Create two subclasses of `Gender` - `Male` and `Female`. Make them disjoint.
5. Set a complete class defintion for `Gender` as `Gender EquivalentTo Male or Female`.
6. Create an object property, `hasGender` , with the domain `Person`, the range `Gender`. Set this as a *Functional* property.
7. Put a restriction on the `Person` class stating that *each and every instance* of the class `Person` holds a `hasGender` property with an instance of the `Gender` class. Define `Person` as a subClass of `hasGender some Gender`. 

The `Person` class is then defined as:  
```
:Person rdf:type owl:Class ;
        rdfs:subClassOf [ rdf:type owl:Restriction ;
                          owl:onProperty :hasGender ;
                          owl:someValuesFrom :Gender
                        ] .
```
And the `Gender` class as: 
```
:Gender rdf:type owl:Class ;
        owl:equivalentClass [ rdf:type owl:Class ;
                              owl:unionOf ( :Female
                                            :Male
                                          )
                            ] ;
        owl:disjointWith :Person .
```

### Task 7: Define `Man` and `Woman` using the definitions for `Person` and `Gender` made earlier. 
1. Create a class `Man`
2. Define it/make it equivalent to a person who is also a male - `Person and (hasGender some Male)`.
3. Do the same for the class `Woman`. 

The class `Man` can now be seen to be defined as: 
```
:Man rdf:type owl:Class ;
     owl:equivalentClass [ owl:intersectionOf ( :Person
                                                [ rdf:type owl:Restriction ;
                                                  owl:onProperty :hasGender ;
                                                  owl:someValuesFrom :Male
                                                ]
                                              ) ;
                           rdf:type owl:Class
                         ] ;
     rdfs:subClassOf :Person .
```

### Task 8: Define `domain` and `range` for certain properties based on classes defined earlier 
1. Add the domain `Person` and the range `Woman` to the property `hasMother`.
2. Similarly for `hasFather`, except the range should be `Man`.
3. Give the property `hasParent` domain and range of `Person`

Run the reasoner again. Check types for the various instances. For example, `Queen_Elizabeth_II` is inferred as a `Woman`. Write DL queries for `Person`, `Man` and `Woman` to see which instances have been inferred as which. For example, `Princess_Margaret_Countess_of_Snowdon` is inferred as a `Person` because *she* appears only on the left (as a subject) of `hasFather`/`hasMother`. 

### Task 9: Inconsistencies 
1. Take an instance like `William_Prince_of_Wales` and make an object property assertion on `hasMother` for it, choosing the object as `King_George_VI`. Run the reasoner. What do you see? The reasoner returns that there is a logical inconsistency somewhere. Click on the *Explain* button to see the explanation.
2. Now, go back to the definition of `hasGender` and remove the contraint on it to be `Functional`. Run the reasoner again. What happens? Go to the instance of (Individuals) `King_George_VI` and check the type description. Reinstate the `Functional` constraint, remove and incorrect assertion and run the reasoner. 
3. Next, define `hasSibling` property as:
```
ObjectProperty: hasSibling
Characteristics: Symmetric, transitive
SubPropertyChain: hasParent o isParentOf
```
Note: First define `isParentOf` to achieve this, as inverse of `hasParent`. 

2. Run the reasoner. Run DL query `hasSibling value Queen_Elizabeth_II`.
3. Add the characteristic `Irreflexive` (object cannot hold the property with itself) to `hasSibling`. Run the reasoner again. What happens? 